In [ ]:
import pandas as pd
import numpy as np
import random
from IPython.display import display
import re


# $ gsutil ls -la "gs://durrant_prot_pred_db/mgnify_scrape_20220505/**" | tee full_scrape.txt
LS_FPATH = '/GeneGraphDB/data/full_scrape.txt'

In [ ]:
with open(LS_FPATH) as f:
    lines = list(f.readlines())
    
lines[0:5]

# Plan

- [ ] how many analyses are there per study?
- [ ] which studies have zero analyses?
- [ ] check no-analysis studies in browser (3). see what the expected number of studies is

In [ ]:
# function that accepts:
#   regex, group labels
# returns:
#   df of filenames matching regex, group labels as columns

ANALYSIS_ARGS = {
    'pattern': re.compile(r'\s*(\d+)\s+2022.*Z\s+gs.*studies\/(\w*)\/samples\/(\w*)/analyses\/(\w+)\/(.*)#.*$'),
    'group_labels': ['size', 'study_id', 'sample_id', 'analysis_id', 'filename'],
}
STUDY_ARGS = {
    'pattern': re.compile(r'\s*(\d+)\s+2022.*Z\s+gs.*studies\/(\w*)\/(study_[^\/]*.json)'),
    'group_labels': ['size', 'study_id', 'fname'],
}


def get_file_df(pattern, group_labels):
    parsed_dicts = []
    for line in lines:
        match_res = pattern.match(line)
        if match_res:
            # idx is + 1 because group 0 is entire regex match
            parsed_dict = {label: match_res.group(group_idx + 1) for group_idx, label in enumerate(group_labels)}
            parsed_dict['full_line'] = match_res.group(0)
            parsed_dicts.append(parsed_dict)
    df = pd.DataFrame(parsed_dicts)
    if 'size' in df:
        df['size'] = df['size'].astype('Int64')
    return df


analysis_df = get_file_df(**ANALYSIS_ARGS)
display(analysis_df.head())

study_df = get_file_df(**STUDY_ARGS)
display(study_df.head())

In [ ]:
df = analysis_df.copy()
df = df[~df['filename'].str.contains('json')]
df = df[df['filename'].str.endswith('.faa.gz')]
# df = df[df['size'] > 10**6]
df.head()

In [ ]:
studies_with_analysis = set(df.groupby('study_id').count().index)
print(len(studies_with_analysis), "studies with analysis")

all_studies = set(study_df['study_id'].unique())

print(len(all_studies), "studies total")

random.sample(list(all_studies - studies_with_analysis), 3)




In [ ]:
study_df[study_df['study_id'] == 'MGYS00005799']['full_line'].values

In [ ]:
print("analysis file sizes in MB's")
(df[['analysis_id', 'size']].groupby('analysis_id').sum() / 10**6).describe()

In [ ]:
df['size'].sum() / 10**9

In [ ]:
df.shape  # compared to 61k analysis.json files.  ~15% of studies have a CDS file?

In [ ]:
x = df.groupby('analysis_id').count()
x['size'].value_counts() / x['size'].index.size

In [ ]:
analysis_df = full_df[full_df['filename'].str.startswith('analysis') & full_df['filename'].str.endswith(".json")]
analysis_df.groupby('study_id').count()

In [ ]:
df.groupby('study_id').count()